In [14]:
!pip install --quiet python-docx langchain-openai langchain_community

In [15]:
!pip install quip

In [16]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [34]:
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders.quip import QuipLoader
from langchain.agents import AgentType, initialize_agent
from langchain.tools import Tool
from docx import Document

In [31]:
# Initialize the language model
llm = ChatOpenAI(temperature=0.7)

In [32]:
# loader = QuipLoader(
#     api_url="https://platform.quip.com", access_token="change_me", request_timeout=60
# )
def quip_documents_generater(topic: str) -> str:
    # Generate content using the LLM
    prompt = f"Write a comprehensive article on the topic: {topic}"
    response = llm.predict(prompt)
    generated_content = response.strip()
    # Create a new Word document
    doc = Document()
    doc.add_heading(topic, 0)
    doc.add_paragraph(generated_content)
    # Save the document
    filename = f"{topic.replace(' ', '_')}.docx"
    doc.save(filename)
    return f"Document '{filename}' has been created and saved successfully."
# documents = loader.load(
#     folder_ids={"123", "456"},
#     thread_ids={"abc", "efg"},
#     include_attachments=False,
#     include_comments=False,
# )

In [33]:
document_creation_tool = Tool(
    name="QuipDocumentCreator",
    func=quip_documents_generater,
    description="Generates a .docx document on a given topic using the LLM."
)

In [35]:
# Initialize the agent with the custom tool
agent_chain = initialize_agent(
    tools=[document_creation_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

<ipython-input-35-4475dcf25c0d>:2: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_chain = initialize_agent(


In [36]:
# Define your topic
topic = "The Impact of Artificial Intelligence on Modern Healthcare"
# Create the document
response = agent_chain.run(f"Create a document on the topic: {topic}")
print(response)

<ipython-input-36-75dc311a144f>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent_chain.run(f"Create a document on the topic: {topic}")




> Entering new AgentExecutor chain...
I should use the QuipDocumentCreator tool to generate a document on the impact of artificial intelligence on modern healthcare.
Action: QuipDocumentCreator
Action Input: "The Impact of Artificial Intelligence on Modern Healthcare"
Observation: Document 'The_Impact_of_Artificial_Intelligence_on_Modern_Healthcare.docx' has been created and saved successfully.
Thought:The document has been successfully created.
Final Answer: The document on the topic "The Impact of Artificial Intelligence on Modern Healthcare" has been generated.

> Finished chain.
The document on the topic "The Impact of Artificial Intelligence on Modern Healthcare" has been generated.
